In [2]:
import requests
import re
from bs4 import BeautifulSoup
import datetime
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd

In [4]:
import requests
import re
from bs4 import BeautifulSoup
import datetime
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd

def generate_date_range(start_date, end_date):
    start_date = datetime.strptime(start_date, "%Y.%m.%d")
    end_date = datetime.strptime(end_date, "%Y.%m.%d")
    
    date_range = []
    while start_date <= end_date:
        date_range.append(start_date.strftime("%Y.%m.%d"))
        start_date += timedelta(days=1)
    
    return date_range

def make_url(search, date, start_pg, end_pg):
    start_dt = date
    start_dt1 = date.replace(".", "")
    urls = []
    for i in range(start_pg, end_pg + 1):
        page = 1 + (i-1) * 10
        url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search}&sort=0&photo=0&field=0&pd=3&ds={start_dt}&de={start_dt}&cluster_rank=27&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{start_dt1}to{start_dt1},a:all&start={page}"
        urls.append(url)
    return urls

def get_articles(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    return [article["href"] for article in articles if "news.naver.com" in article["href"]]

def get_article_data(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    title = soup.select_one("#content > div > div.end_ct_area > h2")
    if title is None:
        title = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")

    date = soup.select_one("div.media_end_head_info_datestamp > div > span")
    if date is None:
        date = soup.select_one("#content > div > div.article_info > span > em")

    content = soup.select_one("#dic_area")
    if content is None:
        content = soup.select_one("#articleBodyContents")

    title = re.sub('<[^>]*>', '', str(title)).strip()
    date = date["data-date-time"] if date is not None else re.sub('<[^>]*>', '', str(date)).strip()
    content = re.sub('<[^>]*>', '', str(content)).replace('\n', ' ').strip()
    
    return title, date, content

search = "가계 경기"
start_date = input("크롤링할 시작 날짜를 입력해주세요. (예: 2022.07.01): ")
end_date = input("크롤링할 마지막 날짜를 입력해주세요. (예: 2022.07.31): ")

date_range = generate_date_range(start_date, end_date)

news_data = []
for date in tqdm(date_range):
    urls = make_url(search, date, 1, 3)
    for url in urls:
        articles = get_articles(url)
        for article in articles:
            title, date, content = get_article_data(article)
            news_data.append((date, title, article, content))

news_df = pd.DataFrame(news_data, columns=['date', 'title', 'link', 'content'])
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
print("크롤링한 기사 개수:", len(news_df))

start_df = start_date.replace('.', '')
end_df = end_date.replace('.', '')
news_df.to_csv(f'{search}_{start_df}_{end_df}.csv', encoding='utf-8-sig', index=False)


100%|██████████| 59/59 [06:31<00:00,  6.63s/it]


크롤링한 기사 개수: 791
